In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb 
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data.head()

In [ ]:
train_data.info()

In [ ]:
fig,axs = plt.subplots(3,3, figsize=(15,12))
sb.histplot(ax=axs[0,0],data=train_data, x = 'Pclass',hue='Survived', bins=10, multiple="dodge", shrink=.8) 
sb.histplot(ax=axs[0,1],data=train_data, x = 'Sex',hue='Survived', bins=10, multiple="dodge", shrink=.8) 
sb.histplot(ax=axs[0,2],data=train_data, x = 'Age',hue='Survived', bins=10, multiple="dodge", shrink=.8) 
sb.histplot(ax=axs[1,0],data=train_data, x = 'SibSp',hue='Survived', bins=10, multiple="dodge", shrink=.8) 
sb.histplot(ax=axs[1,1],data=train_data, x = 'Parch',hue='Survived', bins=10, multiple="dodge", shrink=.8) 
sb.histplot(ax=axs[1,2],data=train_data, x = 'Fare',hue='Survived', bins=10, multiple="dodge", shrink=.8) 
sb.histplot(ax=axs[2,0],data=train_data, x = 'Embarked',hue='Survived', bins=10, multiple="dodge", shrink=.8) 

palette={"female":"C0","male":"C1"}
sb.histplot(ax=axs[2,1],data=train_data, x = 'Age',hue='Sex', bins=10, multiple="dodge", shrink=.8, palette=palette) 
Train_Sur=train_data.query("Survived==1")
sb.histplot(ax=axs[2,2],data=Train_Sur, x = 'Age',hue='Sex', bins=10, multiple="dodge", shrink=.8, palette=palette) 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass","Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

out_test=pd.read_csv('submission.csv')
out_test.head

In [ ]:
Train_Sur=train_data.query("Survived==1")
fig,axs = plt.subplots(3,3, figsize=(15,12))
sb.histplot(ax=axs[0,0],data=Train_Sur, x = 'Pclass', bins=10) 
sb.histplot(ax=axs[0,0],data=train_data, x = 'Pclass', bins=10,alpha=0.3) 

sb.histplot(ax=axs[0,1],data=Train_Sur, x = 'Sex', bins=10)
sb.histplot(ax=axs[0,1],data=train_data, x = 'Sex', bins=10, alpha=0.3) 

sb.histplot(ax=axs[0,2],data=Train_Sur, x = 'Age', bins=10) 
sb.histplot(ax=axs[0,2],data=train_data, x = 'Age', bins=10, alpha=0.3) 

sb.histplot(ax=axs[1,0],data=Train_Sur, x = 'SibSp', bins=10)
sb.histplot(ax=axs[1,0],data=train_data, x = 'SibSp', bins=10, alpha=0.3) 

sb.histplot(ax=axs[1,1],data=Train_Sur, x = 'Parch', bins=10)
sb.histplot(ax=axs[1,1],data=train_data, x = 'Parch', bins=10, alpha=0.3)

sb.histplot(ax=axs[1,2],data=Train_Sur, x = 'Fare', bins=10)
sb.histplot(ax=axs[1,2],data=train_data, x = 'Fare', bins=10, alpha=0.3)

sb.histplot(ax=axs[2,0],data=Train_Sur, x = 'Embarked', bins=10) 
sb.histplot(ax=axs[2,0],data=train_data, x = 'Embarked', bins=10,alpha=0.3)

In [ ]:
sb.set(style="white")
corr = train_data.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(20, 20))
cmap = sb.diverging_palette(220, 10, as_cmap=True)
plt.title('Correlation Matrix', fontsize=18)
sb.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,fmt = '.2f',square=True,
            linewidths=.5, cbar_kws={"shrink": .5}, annot=True)
plt.show()

In [ ]:
# data cleaning
# Find NaN data
train_data_cle=train_data.copy()
print("Nan data percentages are:")
print(train_data.isnull().sum()/len(train_data)*100)
# Fill in the age data
AgeMed=train_data.Age.median()
print("The median of the age is:",AgeMed )
train_data_cle.Age=train_data.Age.fillna(AgeMed)
train_data_cle = train_data_cle[['Sex','Age','SibSp','Parch','Fare','Embarked']]
train_data_cle.dropna(inplace=True);

In [ ]:
from matplotlib.cbook import boxplot_stats
fig, ax= plt.subplots(figsize=(10, 4))
ax.set_xscale('log')
sb.boxplot(x=train_data_cle['Fare'])
outliers = [y for stat in boxplot_stats(train_data_cle['Fare']) for y in stat['fliers']]
print('Outlier percentage of Fare: %.2f %%' % (len(outliers)/len(train_data_cle)*100))

In [ ]:
fig, ax= plt.subplots(figsize=(10, 4))
ax.set_xscale('log')
sb.boxplot(x=train_data_cle['Age'])
outliers = [y for stat in boxplot_stats(train_data_cle['Age']) for y in stat['fliers']]
print('Outlier percentage of age: %.2f %%' % (len(outliers)/len(train_data_cle)*100))

# Conclusions: Age: 7.31% outlier, not very relative 

In [ ]:
fig, ax= plt.subplots(figsize=(10, 4))
sb.boxplot(x=train_data_cle['SibSp'])
outliers = [y for stat in boxplot_stats(train_data_cle['SibSp']) for y in stat['fliers']]
print('Outlier percentage of Sibsp: %.2f %%' % (len(outliers)/len(train_data_cle)*100))


fig, ax= plt.subplots(figsize=(10, 4))
sb.boxplot(x=train_data_cle['Parch'])
outliers = [y for stat in boxplot_stats(train_data_cle['Parch']) for y in stat['fliers']]
print('Outlier percentage of Parch: %.2f %%' % (len(outliers)/len(train_data_cle)*100))
